<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Make-Coarse-Grained--model" data-toc-modified-id="Make-Coarse-Grained--model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Make Coarse-Grained  model</a></span><ul class="toc-item"><li><span><a href="#Parses-atomistic-pdb" data-toc-modified-id="Parses-atomistic-pdb-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Parses atomistic pdb</a></span></li><li><span><a href="#Adds-missing-residues" data-toc-modified-id="Adds-missing-residues-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Adds missing residues</a></span></li><li><span><a href="#Converts-fixed-pdb-to-table" data-toc-modified-id="Converts-fixed-pdb-to-table-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Converts fixed pdb to table</a></span></li><li><span><a href="#Coarse-Grain-AWSEM" data-toc-modified-id="Coarse-Grain-AWSEM-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Coarse Grain AWSEM</a></span></li><li><span><a href="#Merges-AWSEM-and-3SPN2-models" data-toc-modified-id="Merges-AWSEM-and-3SPN2-models-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Merges AWSEM and 3SPN2 models</a></span></li><li><span><a href="#Get-the-sequence-of-the-protein" data-toc-modified-id="Get-the-sequence-of-the-protein-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Get the sequence of the protein</a></span></li><li><span><a href="#Write-the-merged-PDB" data-toc-modified-id="Write-the-merged-PDB-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Write the merged PDB</a></span></li><li><span><a href="#Create-the-system" data-toc-modified-id="Create-the-system-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>Create the system</a></span></li><li><span><a href="#Add-open3SPN2-forcefield" data-toc-modified-id="Add-open3SPN2-forcefield-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>Add open3SPN2 forcefield</a></span></li><li><span><a href="#Add-AWSEM-forcefield" data-toc-modified-id="Add-AWSEM-forcefield-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>Add AWSEM forcefield</a></span></li></ul></li></ul></div>

## Make Coarse-Grained  model

### Parses atomistic pdb

In [1]:
import pandas
import pdbfixer
import simtk.openmm.app
#Reads pdb file to a table
def parsePDB(pdb_file):
    """Parses a pdb file to a table"""
    def pdb_line(line):
        return dict(recname    = str(line[0:6]).strip(),
                    serial     = int(line[6:11]),
                    name       = str(line[12:16]).strip(),
                    altLoc     = str(line[16:17]),
                    resname    = str(line[17:20]).strip(),
                    chainID    = str(line[21:22]),
                    resSeq     = int(line[22:26]),
                    iCode      = str(line[26:27]),
                    x          = float(line[30:38]),
                    y          = float(line[38:46]),
                    z          = float(line[46:54]),
                    occupancy  = float(line[54:60]),
                    tempFactor = float(line[60:66]),
                    element    = str(line[76:78]),
                    charge     = str(line[78:80]))
    with open(pdb_file,'r') as pdb:
        lines=[]
        for line in pdb:
            if len(line)>6 and line[:6] in ['ATOM  ','HETATM']:
                lines+=[pdb_line(line)]
    pdb_atoms=pandas.DataFrame(lines)
    pdb_atoms=pdb_atoms[['recname','serial','name','altLoc',
                         'resname','chainID','resSeq','iCode',
                         'x','y','z','occupancy','tempFactor',
                         'element','charge']]
    return pdb_atoms
temp=parsePDB('1svc.pdb')

### Adds missing residues

In [2]:
pdb_file='1svc.pdb'
#Make coarse_grained atoms
import pdbfixer
def fix_pdb(pdb_file):
    """Fixes a pdb file (adds missing atoms)"""
    fixer = pdbfixer.PDBFixer(filename=pdb_file,)
    fixer.findMissingResidues()
    chains = list(fixer.topology.chains())
    keys = fixer.missingResidues.keys()
    for key in list(keys):
        chain_tmp = chains[key[0]]
        if key[1] == 0 or key[1] == len(list(chain_tmp.residues())):
            del fixer.missingResidues[key]

    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(keepWater=False)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    return fixer
pdb=fix_pdb('1svc.pdb')

### Converts fixed pdb to table

In [3]:
def pdb2table(pdb):
    """ Parses a pdb in the openawsem format and 
    outputs a table that contains all the information 
    on a pdb file """
    cols=['recname','serial','name','altLoc',
          'resname','chainID','resSeq','iCode',
          'x','y','z','occupancy','tempFactor',
          'element','charge']
    data = []
    for atom,pos in zip(pdb.topology.atoms(),pdb.positions):
        residue=atom.residue
        chain=residue.chain
        pos=pos.value_in_unit(simtk.unit.angstrom)
        data += [dict(zip(cols,['ATOM', int(atom.id), atom.name, '',
                                residue.name, chain.id, int(residue.id),'',
                                pos[0], pos[1], pos[2], 0, 0,
                                atom.element.symbol, '']))]
    atom_list = pandas.DataFrame(data)
    atom_list = atom_list[cols]
    atom_list.index = atom_list['serial']
    return atom_list
pdb_table=pdb2table(pdb)


### Coarse Grain AWSEM

In [4]:
def AWSEMCoarseGrained(pdb_table):
    """ Selects AWSEM atoms from a pdb table and returns a table containing only the coarse-grained atoms for AWSEM """
    protein_residues = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS',
                        'GLN', 'GLU', 'GLY', 'HIS', 'ILE',
                        'LEU', 'LYS', 'MET', 'PHE', 'PRO',
                        'SER', 'THR', 'TRP', 'TYR', 'VAL']
    awsem_atoms = ["N", "H", "CA", "C", "O", "CB"]

    #Select coarse grained atoms
    selection=pdb_table[pdb_table.resname.isin(protein_residues) & pdb_table.name.isin(awsem_atoms)].copy()

    #Remove virtual atoms at the end or begining of the chain
    drop_list=[]
    for chain in selection.chainID.unique():
        sel=selection[selection.chainID==chain]
        drop_list+=list(sel[(sel.resSeq == sel.resSeq.min()) & sel['name'].isin(['N','H'])].index)
        drop_list+=list(sel[(sel.resSeq == sel.resSeq.max()) & sel['name'].isin(['C'])].index)
    selection=selection.drop(drop_list)

    #Replace resnames
    selection['real_resname']=selection.resname.copy()
    resname = selection.resname.copy()
    resname[:] = 'NGP'
    resname[selection.resname=='PRO']='IPR'
    resname[selection.resname=='GLY']='IGL'
    selection.resname=resname
    
    #CB element is B
    selection.loc[selection['name']=='CB','element']='B'
    
    #Reorder atoms
    selection.name=pandas.Categorical(selection.name,awsem_atoms)
    selection.sort_values(['chainID','resSeq','name'])

    #Prepare virtual sites
    for c,chain in selection.groupby('chainID'):
        pos_im = 'First'
        for i,residue in chain.groupby('resSeq'):
            idx=dict(zip(residue.name,residue.index))
            pos=dict(zip(residue.name,[residue.loc[i,['x','y','z']] for i in residue.index]))
            if pos_im != 'First':
                if 'N' in idx.keys():
                    selection.loc[idx['N'],['x','y','z']]=0.48318 * pos_im['CA'] + 0.70328*pos['CA'] - 0.18643 *pos_im['O']
                if 'C' in idx.keys():
                    selection.loc[idx['C'],['x','y','z']]=0.44365 * pos_im['CA'] + 0.23520*pos['CA'] + 0.32115 *pos_im['O']
                if 'H' in idx.keys():
                    selection.loc[idx['H'],['x','y','z']]=0.84100 * pos_im['CA'] + 0.89296*pos['CA'] - 0.73389 *pos_im['O']
            pos_im=pos.copy()
    #Renumber
    selection['serial']=range(len(selection))
    
    return selection
awsem_atoms=AWSEMCoarseGrained(pdb_table)
#awsem_atoms

In [5]:
def ff3SPN2CoarseGrained(pdb_table):
    """ Selects DNA atoms from a pdb table and returns a table containing only the coarse-grained atoms for 3SPN2"""
    masses = {"H":1.00794, "C":12.0107, "N":14.0067, "O":15.9994, "P":30.973762,}
    CG = {"O5\'":'P', "C5\'":'S', "C4\'":'S', "O4\'":'S', "C3\'":'S', "O3\'":'P', 
          "C2\'":'S', "C1\'":'S', "O5*":'P', "C5*":'S', "C4*":'S', "O4*":'S', 
          "C3*":'S', "O3*":'P', "C2*":'S', "C1*":'S', "N1":'B', "C2":'B', "O2":'B', 
          "N2":'B', "N3":'B', "C4":'B', "N4":'B', "C5":'B', "C6":'B', "N9":'B', 
          "C8":'B', "O6":'B', "N7":'B', "N6":'B', "O4":'B', "C7":'B', "P":'P', 
          "OP1":'P', "OP2":'P', "O1P":'P', "O2P":'P', "OP3":'P', "HO5'":'P', 
          "H5'":'S', "H5''":'S', "H4'":'S', "H3'":'S', "H2'":'S', "H2''":'S', 
          "H1'":'S', "H8":'B', "H61":'B', "H62":'B','H2':'B', 'H1':'B', 'H21':'B', 
          'H22':'B', 'H3':'B', 'H71':'B', 'H72':'B', 'H73':'B', 'H6':'B', 'H41':'B', 
          'H42':'B', 'H5':'B', "HO3'":'P'}
    cols=['recname','serial','name','altLoc',
          'resname','chainID','resSeq','iCode',
          'x','y','z','occupancy','tempFactor',
          'element','charge']
    temp=pdb_table.copy()
    
    #Select DNA residues
    temp=temp[temp['resname'].isin(['DA','DT','DG','DC'])]
    
    #Groupd the atoms by sugar, phosphate or base
    temp['group']=temp.name.replace(CG)
    temp=temp[temp['group'].isin(['P','S','B'])]
    
    #Calculate center of mass
    temp['mass']=temp.element.replace(masses).astype(float)
    temp[['x','y','z']]=(temp[['x','y','z']].T*temp['mass']).T[['x','y','z']]
    temp=temp[temp['element']!='H'] #Exclude hydrogens
    Coarse=temp.groupby(['chainID','resSeq','resname','group']).sum().reset_index()
    Coarse[['x','y','z']]=(Coarse[['x','y','z']].T/Coarse['mass']).T[['x','y','z']]
    
    #Set pdb columns
    Coarse['recname']='ATOM'
    Coarse['name']=Coarse['group']
    Coarse['altLoc']=''
    Coarse['iCode']=''
    Coarse['charge']=''
    #Change name of base to real base
    mask=(Coarse.name=='B')
    Coarse.loc[mask,'name']=Coarse[mask].resname.str[-1] #takes last letter from the residue name
    #Set element (depends on base)
    Coarse['element']=Coarse['name'].replace({'P':'P','S':'H','A':'N','T':'S','G':'C','C':'O'})
    #Remove P from the beggining
    drop_list=[]
    for chain in Coarse.chainID.unique():
        sel=Coarse[Coarse.chainID==chain]
        drop_list+=list(sel[(sel.resSeq == sel.resSeq.min()) & sel['name'].isin(['P'])].index)
    Coarse=Coarse.drop(drop_list)
    #Renumber
    Coarse['serial']=range(len(Coarse))
    return Coarse[cols]
dna_atoms=ff3SPN2CoarseGrained(pdb_table)

### Merges AWSEM and 3SPN2 models

In [6]:
#Merge models
Coarse=pandas.concat([awsem_atoms,dna_atoms],sort=False)
Coarse.index=range(len(Coarse))
Coarse.serial=list(Coarse.index)
Coarse.tail()

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge,real_resname
3747,ATOM,3747,P,,DG,D,18,,20.302695,26.282242,57.231301,0,0,P,,NaN
3748,ATOM,3748,S,,DG,D,18,,22.042475,26.441067,59.352817,0,0,H,,NaN
3749,ATOM,3749,A,,DA,D,19,,28.629237,29.572005,64.840530,0,0,N,,NaN
3750,ATOM,3750,P,,DA,D,19,,20.919063,27.574945,62.756124,0,0,P,,NaN
3751,ATOM,3751,S,,DA,D,19,,24.309713,28.022201,63.836948,0,0,H,,NaN


### Get the sequence of the protein

In [7]:
_AWSEMresidues=['IPR','IGL','NGP']
protein_data=Coarse[Coarse.resname.isin(_AWSEMresidues)].copy()
resix = (protein_data.chainID + '_' + protein_data.resSeq.astype(str))
res_unique = resix.unique()
protein_data['resID'] = resix.replace(dict(zip(res_unique, range(len(res_unique)))))
protein_sequence=[r.iloc[0]['real_resname'] for i, r in protein_data.groupby('resID')]      


### Write the merged PDB

In [8]:
def writePDB(atoms,pdb_file):
    with open(pdb_file, 'w+') as pdb:
        for i, atom in atoms.iterrows():
            pdb_line = f'{atom.recname:<6}{atom.serial:>5} {atom["name"]:^4}{atom.altLoc:1}'+\
                       f'{atom.resname:<3} {atom.chainID:1}{atom.resSeq:>4}{atom.iCode:1}   '+\
                       f'{atom.x:>8.3f}{atom.y:>8.3f}{atom.z:>8.3f}' +\
                       f'{atom.occupancy:>6.2f}{atom.occupancy:>6.2f}'+' ' * 10 +\
                       f'{atom.element:>2}{atom.charge:>2}'
            assert len(pdb_line) == 80, f'An item in the atom table is longer than expected ({len(pdb_line)})\n{pdb_line}'
            pdb.write(pdb_line + '\n')
writePDB(Coarse,'clean.pdb')
#if 'C' in r_im:
#    r_im['C'].set_coord(0.44365*r_im['CA'].get_coord()+ 0.23520*r_i['CA'].get_coord()+ 0.32115 *r_im['O'].get_coord())
#if 'H' in r_i:
#    r_i['H'].set_coord( 0.84100*r_im['CA'].get_coord()+ 0.89296*r_i['CA'].get_coord()- 0.73389 *r_im['O'].get_coord())

### Create the system

In [9]:
import simtk.openmm
pdb=simtk.openmm.app.PDBFile('clean.pdb')
top=pdb.topology
coord=pdb.positions
forcefield=simtk.openmm.app.ForceField('awsem.xml','3SPN2.xml')
s=forcefield.createSystem(top)


In [10]:
import numpy as np
size=2*200*simtk.openmm.unit.nanometer
centered=np.array(coord)-np.array(coord).mean(axis=0)+np.array(size/2)
s.setDefaultPeriodicBoxVectors(*np.diag([size]*3))

### Add open3SPN2 forcefield

In [11]:
import sys
sys.path.insert(0, '../../../open3SPN2')
import ff3SPN2
import importlib
importlib.reload(ff3SPN2)

<module 'ff3SPN2' from '../../../open3SPN2/ff3SPN2.py'>

In [12]:
dna=ff3SPN2.DNA.fromCoarsePDB('clean.pdb')

/home/cab22/Programs/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/cab22/Programs/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
import sys
sys.path.insert(0, '../../../open3SPN2')
import ff3SPN2
import importlib
importlib.reload(ff3SPN2)
dna=ff3SPN2.DNA.fromCoarsePDB('clean.pdb')
keepCMMotionRemover=True
#Clear Forces
j=0
for i, f in enumerate(s.getForces()):
    if keepCMMotionRemover and i == 0 and f.__class__ == simtk.openmm.CMMotionRemover:
        # print('Kept ', f.__class__)
        j += 1
        continue
    else:
        # print('Removed ', f.__class__)
        s.removeForce(j)
if keepCMMotionRemover == False:
    assert len(s.getForces()) == 0, 'Not all the forces were removed'
else:
    assert len(s.getForces()) <= 1, 'Not all the forces were removed'


#Add forces
forces = dict(Bond=ff3SPN2.Bond3SPN2,
              Angle=ff3SPN2.Angle3SPN2,
              Stacking=ff3SPN2.Stacking3SPN2,
              Dihedral=ff3SPN2.Dihedral3SPN2,
              BasePair=ff3SPN2.BasePair3SPN2,
              CrossStacking=ff3SPN2.CrossStacking3SPN2,
              Exclusion=ff3SPN2.Exclusion3SPN2,
              Electrostatics=ff3SPN2.Electrostatics3SPN2
             )
for force_name in forces:
    print(force_name)
    force = forces[force_name](dna)
    if force_name == 'CrossStacking':
        force.addForce(s)
    else:
        s.addForce(force)

/home/cab22/Programs/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/cab22/Programs/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Bond
Angle
Stacking
Dihedral
BasePair
CrossStacking
Exclusion
Electrostatics
77.8 0.974998211
0.94856634121177 nm 3.5975908035210326e-70 mol C**2 F/m


In [14]:
import simtk.openmm.app
import simtk.openmm
import simtk.unit as unit
import configparser
import numpy as np
import itertools
import scipy.spatial.distance as sdist
import os

class O(object):
    pass
self=O()

In [16]:
temperature=300 * simtk.openmm.unit.kelvin
platform_name='OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1E-4 / unit.picosecond, 2 * unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top,s, integrator,platform)
simulation.context.setPositions(centered)
energy_unit=unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)
simulation.reporters.append(simtk.openmm.app.PDBReporter(f'output.pdb', 1000),)
simulation.reporters.append(simtk.openmm.app.DCDReporter(f'output.dcd', 1000),)
#sim_out=open('sim_out.txt','w+')
simulation.reporters.append(simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,potentialEnergy=True, temperature=True,separator='\t',))
simulation.reporters.append(simtk.openmm.app.StateDataReporter('sim.log', 1000, step=True,time=True,totalEnergy=True,
                                              kineticEnergy=True,potentialEnergy=True, temperature=True))

simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(100000)

-739.2152099609375
#"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"
1000	2.0000000000000013	-654.27490234375	294.80621900678585
2000	3.999999999999781	-669.3508911132812	295.4667890607743
3000	5.999999999999561	-665.025390625	295.6120538938855
4000	7.999999999999341	-679.1336669921875	296.23911757586296


KeyboardInterrupt: 

### Add AWSEM forcefield

In [17]:
sys.path.insert(0, '../../')

In [18]:
import functionTerms.basicTerms as bt

In [19]:
import ffAWSEM
importlib.reload(ffAWSEM)
protein=ffAWSEM.Protein.fromCoarsePDB('clean.pdb',sequence=protein_sequence)

###Extra steps taken
- Create ssweight
- Copy parameters

In [20]:
import functionTerms
'''
forces = [
#    q_value(oa, "crystal_structure-cleaned.pdb"),
    con_term(oa),
    chain_term(oa),
    chi_term(oa),
    excl_term(oa),
    rama_term(oa),
    rama_proline_term(oa),
    rama_ssweight_term(oa),
    contact_term(oa, z_dependent=False),
    beta_term_1(oa),
    beta_term_2(oa),
    beta_term_3(oa),
    pap_term_1(oa),
    pap_term_2(oa),
    # pap_term_old(oa),
    # fragment_memory_term(oa, frag_location_pre="./"),
    # er_term(oa),
    # tbm_q_term(oa, k_tbm_q=2000),
    # membrane_term(oa),
] '''
if False:
    #Clear Forces
    j=0
    for i, f in enumerate(s.getForces()):
        if keepCMMotionRemover and i == 0 and f.__class__ == simtk.openmm.CMMotionRemover:
            # print('Kept ', f.__class__)
            j += 1
            continue
        else:
            # print('Removed ', f.__class__)
            s.removeForce(j)
    if keepCMMotionRemover == False:
        assert len(s.getForces()) == 0, 'Not all the forces were removed'
    else:
        assert len(s.getForces()) <= 1, 'Not all the forces were removed'

#Add forces
forces = dict(Connectivity=functionTerms.basicTerms.con_term,
              Chain=functionTerms.basicTerms.chain_term,
              Chi=functionTerms.basicTerms.chi_term,
              Excl=functionTerms.basicTerms.excl_term,
              rama=functionTerms.basicTerms.rama_term,
              rama_pro=functionTerms.basicTerms.rama_proline_term,
              rama_ss=functionTerms.basicTerms.rama_ssweight_term,
#              contact=functionTerms.contactTerms.contact_term,
#              beta1=  functionTerms.hydrogenBondTerms.beta_term_1,
#              beta2=  functionTerms.hydrogenBondTerms.beta_term_2,
#              beta3=  functionTerms.hydrogenBondTerms.beta_term_3,
#              pap1=  functionTerms.hydrogenBondTerms.pap_term_1,
#              pap2=  functionTerms.hydrogenBondTerms.pap_term_2,
             )
protein.setup_virtual_sites(s)
for force_name in forces:
    print(force_name)
    force = forces[force_name](protein)
    s.addForce(force)


Connectivity
Chain
Chi
Excl
rama
rama_pro
rama_ss


In [22]:
temperature=300 * simtk.openmm.unit.kelvin
platform_name='OpenCL'

integrator = simtk.openmm.LangevinIntegrator(temperature, 1E-4 / simtk.openmm.unit.picosecond, 2 * simtk.openmm.unit.femtoseconds)
platform = simtk.openmm.Platform.getPlatformByName(platform_name)
simulation = simtk.openmm.app.Simulation(top,s, integrator,
                                              platform)
simulation.context.setPositions(centered)
energy_unit=simtk.openmm.unit.kilojoule_per_mole
state = simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(energy_unit)
print(energy)
simulation.reporters.append(simtk.openmm.app.PDBReporter(f'output.pdb', 1000),)
simulation.reporters.append(simtk.openmm.app.DCDReporter(f'output.dcd', 1000),)
#sim_out=open('sim_out.txt','w+')
simulation.reporters.append(simtk.openmm.app.StateDataReporter(sys.stdout, 1000, step=True,time=True,potentialEnergy=True, temperature=True,separator='\t',))
simulation.reporters.append(simtk.openmm.app.StateDataReporter('sim.log', 1000, step=True,time=True,totalEnergy=True,
                                              kineticEnergy=True,potentialEnergy=True, temperature=True))

simulation.minimizeEnergy()
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(100000)

495700.125
#"Step"	"Time (ps)"	"Potential Energy (kJ/mole)"	"Temperature (K)"
1000	2.0000000000000013	-7832.6123046875	169.09068343924494


KeyboardInterrupt: 